In [1]:
import warnings
warnings.filterwarnings('ignore') 
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter 
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression 
from imblearn.over_sampling import RandomOverSampler 
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import ClusterCentroids 
from imblearn.combine import SMOTEENN 
from imblearn.ensemble import BalancedRandomForestClassifier   
from imblearn.ensemble import EasyEnsembleClassifier

In [2]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [3]:
file_path = Path('./LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns 
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove `Issued` 
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# make int_rate numeric
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# make the target column values to low_risk and high_risk
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [4]:
# features
X = df.drop(columns=['loan_status'])
X = pd.get_dummies(X, drop_first=False)

#target
y = df['loan_status']

In [5]:
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [7]:
ros = RandomOverSampler() 
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [8]:
y_resampled.value_counts()

low_risk     51352
high_risk    51352
Name: loan_status, dtype: int64

In [9]:
classifier=LogisticRegression() 
classifier.fit(X_resampled, y_resampled)

LogisticRegression()

In [10]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'high_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [11]:
balanced_accuracy_score(y_test,y_predict)

0.6139061121384628

In [12]:
confusion_matrix(y_test,y_predict)

array([[   51,    36],
       [ 6135, 10983]])

In [13]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.59      0.64      0.02      0.61      0.37        87
   low_risk       1.00      0.64      0.59      0.78      0.61      0.38     17118

avg / total       0.99      0.64      0.59      0.78      0.61      0.38     17205



In [14]:
#SMOTE

In [15]:
smo = SMOTE() 
X_resampled, y_resampled = smo.fit_resample(X_train, y_train)

In [16]:
classifier=LogisticRegression() 
classifier.fit(X_resampled, y_resampled)

LogisticRegression()

In [17]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'high_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [18]:
balanced_accuracy_score(y_test, y_predict)

0.6014258030466015

In [19]:
confusion_matrix(y_test,y_predict)

array([[   48,    39],
       [ 5972, 11146]])

In [20]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.55      0.65      0.02      0.60      0.36        87
   low_risk       1.00      0.65      0.55      0.79      0.60      0.36     17118

avg / total       0.99      0.65      0.55      0.78      0.60      0.36     17205



In [ ]:
#SMOTEENN

In [38]:
smoteenn = SMOTEENN()
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)

In [39]:
classifier=LogisticRegression() 
classifier.fit(X_resampled, y_resampled)

LogisticRegression()

In [40]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'high_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [41]:
balanced_accuracy_score(y_test, y_predict)

0.5992592995475623

In [42]:
confusion_matrix(y_test,y_predict)

array([[  57,   30],
       [7817, 9301]])

In [43]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.66      0.54      0.01      0.60      0.36        87
   low_risk       1.00      0.54      0.66      0.70      0.60      0.35     17118

avg / total       0.99      0.54      0.65      0.70      0.60      0.35     17205



In [ ]:
#BalancedRandomForestClassifier

In [24]:
classifier = BalancedRandomForestClassifier()

In [25]:
classifier.fit(X_train, y_train)

BalancedRandomForestClassifier()

In [26]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [27]:
balanced_accuracy_score(y_test, y_predict)

0.8276325384451132

In [28]:
confusion_matrix(y_test,y_predict)

array([[   67,    20],
       [ 1966, 15152]])

In [29]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.77      0.89      0.06      0.83      0.67        87
   low_risk       1.00      0.89      0.77      0.94      0.83      0.69     17118

avg / total       0.99      0.88      0.77      0.93      0.83      0.69     17205



In [30]:
df = pd.DataFrame({"name":classifier.feature_names_in_, "score": classifier.feature_importances_})
df=df.sort_values('score', ascending=False) 
print(df.to_string())

                                   name     score
15                      total_rec_prncp  0.073857
20                      last_pymnt_amnt  0.064230
13                          total_pymnt  0.057526
14                      total_pymnt_inv  0.054422
16                        total_rec_int  0.052557
84                     issue_d_Jan-2019  0.026708
1                              int_rate  0.018860
12                        out_prncp_inv  0.018686
4                                   dti  0.018504
72                      tot_hi_cred_lim  0.017431
2                           installment  0.017351
11                            out_prncp  0.017007
25                          tot_cur_bal  0.016756
35                           max_bal_bc  0.016414
48                 mo_sin_old_rev_tl_op  0.016194
85                     issue_d_Mar-2019  0.016138
3                            annual_inc  0.015427
32                              il_util  0.015225
44                              bc_util  0.014658


In [31]:
# EasyEnsembleClassifier

In [32]:
classifier = EasyEnsembleClassifier(n_estimators = 100)

In [33]:
classifier.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100)

In [34]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [35]:
balanced_accuracy_score(y_test, y_predict)

0.9284066781891214

In [36]:
confusion_matrix(y_test,y_predict)

array([[   79,     8],
       [  877, 16241]])

In [37]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.95      0.15      0.93      0.86        87
   low_risk       1.00      0.95      0.91      0.97      0.93      0.87     17118

avg / total       0.99      0.95      0.91      0.97      0.93      0.86     17205

